## <span style="color:DeepSkyBlue;">Walking on Thick Ice</span>: Measuring Antarctic Ice Warming using IceCube Thermistor Data
### Collaborators: Diyaselis Delgado, Alexis Mulski, Josh Villareal

In [ ]:
import icetemp

The generative model used to analyze the data involves an uniform prior.

$$ \Delta T \sim \text{uniform}(0.01^{\circ}\text{C},0.1^{\circ}\text{C})$$

Noise sources considered are directly related to the thermistor technical information. 

The calculatations of the likelihood function from your statistical model is given by: 

In [ ]:
likelihood = icetemp.likehood()